# Example 9: Multi-asset trend following strategy

## Pre-requisites
### 1. If you have not opened the notebook in Colab, select the button below
<a href="https://githubtocolab.com/SIGTechnologies/sigtech-python/blob/master/examples/notebooks/09_Multi-Asset_Trend_Following.ipynb">
    <img src="https://sigtech.com/wp-content/uploads/2023/08/grey_google_colab.svg"></a>

### 2. Enter your API key
After pasting in your API key, you need to run the cell. In Colab, hover your cursor over an individual code cell and click play to run it.
>**Tip**!\
>After pasting in your API key, you can press `CTRL-F9` (Windows) or `⌘-F9` (Mac) to run the entire notebook at once.

In [ ]:
# Install our Python SDK
%pip install sigtech
%pip install empyrical 

# Import OS and our Python SDK
import sigtech.api as sig
import os

# Define your API key as a string. Remember to delete it before sharing your notebook with others. Replace 
# <YOUR_API_KEY> with the API key you have generated. e.g. os.environ['SIGTECH_API_KEY'] = 'sig_A1B2C3D4E5f6g7h8i9'
os.environ['SIGTECH_API_KEY'] = '<YOUR_API_KEY>'

### 3. Set up your Colab environment

In [ ]:
# Import any additional Python libraries you require.
import datetime as dtm
import pandas as pd
import numpy as np
import empyrical as ep 
import matplotlib.pyplot as plt

# Set any parameters 
plt.rcParams['figure.figsize'] = [16, 8]

### 4. Create a session
After installing our Python SDK, defining your API key, importing any additional Python libraries or functions you require, and setting any default parameters, initialize your session.

In [ ]:
sig.init()

## Introduction to trend following strategies

A trend-following strategy aims to capitalize on market momentum by buying assets during upward trends and selling or shorting them during downward trends. The strategy relies on technical indicators like moving averages to identify the direction of market trends and make trading decisions accordingly.

## Our strategy

- We will fetch historical prices for the E-mini S&P 500 index and the commodity and evaluate how a basket of these instruments would perform.
- Next, we will calculate short, medium, and long-term momentum for these instruments and normalize momentum by asset volatility, then calculate a weighted sum of normalized momentum for different periods.
- Using this, we will generate trading signals based on composite trend scores.
- We will calculate position sizes based on trading signals and volatility and execute the strategy using a `SignalStrategy` object.
- We will evaluate the performance of the strategy using the empyrical library. 

Our strategy aims to capture momentum across a universe of equities and bonds by calculating short, medium, and long-term momentum indicators. It dynamically adjusts asset allocations based on these momentum signals, aiming to outperform a static bond-equity 60/40 portfolio.

1. Evaluate the historical performance of the static strategy

In [ ]:
universe = [
    ('ES', 'INDEX'), # E-mini S&P 500 futures index
    ('TY', 'COMDTY'),# US 10-year treasury bonds
]

# Create a list of RollingFutureStrategy objects for each asset in the universe
assets = [sig.RollingFutureStrategy(contract_code = x, contract_sector = y) for x, y in universe]

In [ ]:
# Retrieve historical data for each RollingFutureStrategy object and store it in a DataFrame
df = pd.DataFrame({x.name:x.history() for x in assets})

In [ ]:
# Plot the historical data stored in the DataFrame
df.plot(legend = True)

Next, we create a basket of our instruments using the `BasketStrategy` class. 

In [ ]:
# Create a bond-equity 60-40 strategy using a BasketStrategy object
bond_equity_60_40 = sig.BasketStrategy(
    start_date = dtm.date(2018,1,10), 
    constituent_names = list(assets), 
    weights = [0.4,0.6]
    )

# Plot the historical performance of the bond-equity 60-40 strategy
bond_equity_60_40.history().plot()


## 2. Calculate momentum and generate trading signals
In this step, we:

- Define and calculate momentum for short, medium, and long-term periods.
- Calculate rolling volatility and normalizes momentum by this volatility. This step is essential for risk management, ensuring that assets with higher volatility don't overly influence the portfolio.
- Compute a composite trend score, a weighted sum of normalized momentum indicators. This step aims to capture trends at multiple time horizons and is crucial for making well-informed trading decisions.
- Generate trading signals and calculate position sizes based on composite trend scores and volatility. These steps translate the analytical measures into actionable trading decisions, crucial for executing the strategy.
- Clean up the DataFrame to only include position sizes and drop missing values, preparing the data for backtesting using the `SignalStrategy` class.

In [ ]:
# Define assets based on the DataFrame columns
assets = df.columns

# Define the periods for short-term, medium-term, and long-term momentum
N = 20  # Short-term period
M = 60  # Medium-term period
L = 120  # Long-term period

In [ ]:
# 1. Calculate momentum for each asset and for each time period (N, M, L)
for asset in assets:
    df[f'{asset}_momentum_short'] = df[asset] - df[asset].shift(N)
    df[f'{asset}_momentum_medium'] = df[asset] - df[asset].shift(M)
    df[f'{asset}_momentum_long'] = df[asset] - df[asset].shift(L)


In [ ]:
# 2. Define the period for volatility calculation
K = 20  
returns = df.pct_change()

# Calculate the rolling volatility for each asset
for asset in assets:
    df[f'{asset}_volatility'] = returns[asset].rolling(window=K).std()

# Normalize momentum by dividing it by volatility
for asset in assets:
    df[f'{asset}_normalized_momentum_short'] = df[f'{asset}_momentum_short'] / df[f'{asset}_volatility']
    df[f'{asset}_normalized_momentum_medium'] = df[f'{asset}_momentum_medium'] / df[f'{asset}_volatility']
    df[f'{asset}_normalized_momentum_long'] = df[f'{asset}_momentum_long'] / df[f'{asset}_volatility']


In [ ]:
# 3. Composite Trend Score
# Define example weights for composite trend score calculation
w1, w2, w3 = 0.2, 0.3, 0.5  

# Calculate composite trend score for each asset
for asset in assets:
    df[f'{asset}_composite_score'] = w1 * df[f'{asset}_normalized_momentum_short'] + w2 * df[f'{asset}_normalized_momentum_medium'] + w3 * df[f'{asset}_normalized_momentum_long']

In [ ]:
# 4. Signal Generation
# Define thresholds for signal generation
positive_threshold = 0.5  
negative_threshold = -0.5  

# Generate trading signals based on composite trend score
for asset in assets:
    df[f'{asset}_signal'] = np.where(df[f'{asset}_composite_score'] > positive_threshold, 1, 
                                     np.where(df[f'{asset}_composite_score'] < negative_threshold, -1, 0))


In [ ]:
# 5. Position Sizing
# Calculate position size based on trading signal and volatility
for asset in assets:
    df[f'{asset}_position_size'] = df[f'{asset}_signal'] / df[f'{asset}_volatility']

## 3. Create a `SignalStrategy` to backtest trading this portfolio using momentum indicators

First, we need to create a signal dataframe to use as an input for the `SignalStrategy` class. A signal dataframe is a pandas DataFrame where the column headers are the instrument names and the values are the signals for each of the instruments. These signals can be either a number of units *or* a weight.

In [ ]:
# Keep only position sizes and normalize weights

positions = df[[col for col in df.columns if 'position_size' in col]]

positions.columns = assets

positions = positions.divide(positions.abs().sum(axis=1), axis=0)

positions.tail()

In [ ]:
signal_df = positions.copy().dropna()

In [ ]:
signal_df

Now, we can create our `SignalStrategy`

In [ ]:
s = sig.SignalStrategy(
        currency = 'USD',
        start_date = dtm.date(2018,1,10),
        signal_input = signal_df,
        rebalance_frequency = 'EOM' # Rebalances monthly at the end of the month.
)

After creating it, we can view its historical performance.

In [ ]:
s.history().plot(legend = True, label = 'Equity - Bond Trend Portfolio')

In [ ]:
def calculate_metrics(returns, factor_returns, strategy_name):
    # Calculate metrics
    beta = ep.beta(returns, factor_returns)
    sharpe_ratio = ep.sharpe_ratio(returns)
    sortino_ratio = ep.sortino_ratio(returns)
    annual_volatility = ep.annual_volatility(returns)
    annual_return = ep.annual_return(returns)
    max_drawdown = ep.max_drawdown(returns)
    omega_ratio = ep.omega_ratio(returns)
    tail_ratio = ep.tail_ratio(returns)
    cagr = ep.cagr(returns)

    # Create a DataFrame
    metrics = pd.DataFrame({
        'Beta': [beta],
        'Sharpe Ratio': [sharpe_ratio],
        'Sortino Ratio': [sortino_ratio],
        'Annual Volatility': [annual_volatility],
        'Annual Return': [annual_return],
        'Max Drawdown': [max_drawdown],
        'Omega Ratio': [omega_ratio],
        'Tail Ratio': [tail_ratio],
        'CAGR': [cagr]
    }, index=[strategy_name]).transpose()

    return metrics

In [ ]:
# Assuming 's.history()' and 'es.history()' are your strategy's and ES's return series respectively
strategy_returns = s.history().pct_change().dropna()
benchmark_returns = bond_equity_60_40.history().pct_change().dropna()

# Calculate the metrics
strategy_metrics = calculate_metrics(strategy_returns, benchmark_returns, 'Bond Equity Trend Following')
benchmark_metrics = calculate_metrics(benchmark_returns, benchmark_returns, 'Bond Equity 60/40')

# Concatenate the metrics
all_metrics = pd.concat([strategy_metrics, benchmark_metrics], axis=1)

all_metrics